<a href="https://colab.research.google.com/github/DigitalTibetan/DigitalTibetan/blob/main/docs/computational_dharma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computational dharma, really?

some first tests...

## Conversion from Wylie to Unicode Tibetan

In [3]:
!pip install pyewts

  Created wheel for pyewts: filename=pyewts-0.1.3-py3-none-any.whl size=13542 sha256=74f7ebfa2caa6e7000066f976f2a14cc2dd040c421060439f0cfcb3c5396afe1
  Stored in directory: /root/.cache/pip/wheels/5c/01/8a/f2ed100460b6feae43209fbdade79c591e03170570963ba3b2
Successfully built pyewts


In [4]:
import pyewts

In [5]:
converter=pyewts.pyewts()

In [7]:
print(converter.toUnicode("oM AH hUM:"))

ཨོཾ་ཨཱཿ་ཧཱུཾ༔
